In [ ]:
!
!pip install mediapipe opencv-python

import cv2
import mediapipe as mp
import numpy as np
import os
from google.colab import files

In [2]:
class RowingCoachAI:
    def __init__(self):
        self.mp_pose = mp.solutions.pose
        # Increased confidence thresholds for real video to reduce jitter
        self.pose = self.mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6)
        self.mp_drawing = mp.solutions.drawing_utils

        # State variables
        self.previous_landmarks = None

    def calculate_angle(self, a, b, c):
        """ Calculates angle between three points (a, b, c) at joint b """
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)

        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)

        if angle > 180.0:
            angle = 360-angle

        return angle

    def analyze_form(self, landmarks, image_shape):
        """ The Core Logic: Geometry and Physics checks """
        h, w, _ = image_shape

        # Helper to get pixel coordinates
        def get_coords(landmark_name):
            lm = landmarks[self.mp_pose.PoseLandmark[landmark_name].value]
            return [lm.x * w, lm.y * h]

        # Key Joints
        # Note: We focus on the LEFT side (standard view).
        # If video is from the right, change LEFT_ to RIGHT_
        shoulder = get_coords("LEFT_SHOULDER")
        hip = get_coords("LEFT_HIP")
        knee = get_coords("LEFT_KNEE")
        ankle = get_coords("LEFT_ANKLE")
        wrist = get_coords("LEFT_WRIST")

        feedback = []
        color_overlay = (0, 255, 0) # Green (Good)

        # --- 1. COMPRESSION CHECK (Shin Angle) ---
        # Create virtual point above ankle
        vertical_point = [ankle[0], ankle[1] - 100]
        shin_angle = self.calculate_angle(vertical_point, ankle, knee)

        # If shins go past vertical (angle < 90 implies leaning forward too much relative to vertical)
        # We use a range to ensure it's detecting the catch phase
        if shin_angle > 15 and shin_angle < 85:
             feedback.append(f"OVER COMPRESSION: {int(shin_angle)} deg")
             color_overlay = (0, 0, 255) # Red

        # --- 2. BODY SWING CHECK (Layback) ---
        vertical_hip = [hip[0], hip[1] - 100]
        torso_angle = self.calculate_angle(vertical_hip, hip, shoulder)

        # If leaning back too far at the finish
        if torso_angle > 40:
            feedback.append(f"EXCESSIVE LAYBACK: {int(torso_angle)} deg")
            color_overlay = (0, 165, 255) # Orange

        # --- 3. SEQUENCING CHECK (Shooting the Slide) ---
        if self.previous_landmarks is not None:
            prev_hip = self.previous_landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value]
            prev_wrist = self.previous_landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST.value]

            # Velocity (pixels per frame)
            hip_velocity = abs(hip[0] - (prev_hip.x * w))
            wrist_velocity = abs(wrist[0] - (prev_wrist.x * w))

            # If seat moves fast but handle moves slow
            if hip_velocity > 5.0 and wrist_velocity < 2.0:
                 feedback.append("FAULT: SHOOTING SLIDE")
                 color_overlay = (0, 0, 255)

        self.previous_landmarks = landmarks
        return feedback, color_overlay

    def process_video(self, input_path, output_path):
        cap = cv2.VideoCapture(input_path)

        if not cap.isOpened():
            print(f"Error opening video file: {input_path}")
            return

        # Get video properties
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        # Create Video Writer
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

        print(f"Processing {input_path} ({fps} FPS)...")

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # MediaPipe requires RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = self.pose.process(image)

            # BGR for OpenCV
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                # 1. Analyze Form
                warnings, color = self.analyze_form(landmarks, image.shape)

                # 2. Draw Skeleton
                self.mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    self.mp_pose.POSE_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                    self.mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
                )

                # 3. Draw Feedback Text
                y_pos = 50
                if warnings:
                    for warn in warnings:
                        cv2.putText(image, warn, (20, y_pos),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        y_pos += 40
                else:
                    # If no major faults, show Green "Good Form"
                    cv2.putText(image, "FORM: GOOD", (20, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            out.write(image)
            frame_count += 1

        cap.release()
        out.release()
        print(f"Finished: {output_path}")

In [3]:
coach = RowingCoachAI()

print("Please upload your rowing video(s). MP4 format recommended.")
print("Click the 'Choose Files' button below:")


uploaded = files.upload()

if len(uploaded) == 0:
    print("No files uploaded.")
else:
    for filename in uploaded.keys():
        print(f"\n--- Starting Analysis for {filename} ---")

        # Define output filename
        output_filename = f"analyzed_{filename}"

        # Run the AI
        coach.process_video(filename, output_filename)

        # 3. DOWNLOAD STEP
        print(f"Downloading {output_filename}...")
        files.download(output_filename)

Please upload your rowing video(s). MP4 format recommended.
Click the 'Choose Files' button below:


No files uploaded.
